<a href="https://colab.research.google.com/github/marinaniet0/TransformerGrooveTap2Drum/blob/main/Transformer_Groove_Tap2Drum_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/marinaniet0/TransformerGrooveTap2Drum/Transformer-Groove-Tap2Drum-Demo.ipynb)